In [0]:
from pyspark.sql import SparkSession, functions as f

file_path_hospital1 = "abfss://bronze@emrgen2.dfs.core.windows.net/Hosptial1/departments"
df_hospital1 = spark.read.parquet(file_path_hospital1)
file_path_hospital10 = "abfss://bronze@emrgen2.dfs.core.windows.net/hospital10/departments"
df_hospital10 = spark.read.parquet(file_path_hospital10)

df_merged = df_hospital10.union(df_hospital1)
df_merged = df_merged.withColumn("SRC_Dept_id", f.col("deptid")) \
                     .withColumn("Dept_id", f.concat(f.col("deptid"),f.lit('-'), f.col("datasource"))) \
                     .drop("deptid")


df_merged.createOrReplaceTempView("departments")


In [0]:
%sql
select * from departments
limit 5

In [0]:
%sql
drop table silver.departments;
CREATE TABLE IF NOT EXISTS silver.departments (
Dept_Id string,
SRC_Dept_Id string,
Name string,
datasource string,
is_quarantined boolean
)
USING DELTA
LOCATION "abfss://silver@emrgen2.dfs.core.windows.net/departments";



In [0]:
%sql
truncate table silver.departments;


In [0]:
%sql
insert into silver.departments
SELECT 
Dept_Id,
SRC_Dept_Id,
Name,
Datasource,
    CASE 
        WHEN SRC_Dept_Id IS NULL OR Name IS NULL THEN TRUE
        ELSE FALSE
    END AS is_quarantined
FROM departments

In [0]:
%sql
select * from silver.departments
